In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
# Import API key
import gmaps
# Google developer API key
from datetime import date
from config import gkey
from pprint import pprint
# Configure gmaps
gmaps.configure(api_key=gkey)


In [3]:
#data provided by emily and harlan https://github.com/NABSA/gbfs/blob/master/systems.csv
bigdata_df=pd.read_csv("big_data.csv")
bigdata_df


,Country Code,Name,Location,System ID,URL,Auto-Discovery URL
0,AE,ADCB Bikeshare,"Abu Dhabi, AE",ABU,https://www.bikeshare.ae/,https://api-core.bikeshare.ae/gbfs/gbfs.json
1,AE,Careem BIKE,"Dubai, AE",careem_bike,https://www.careem.com/en-ae/careem-bike/,https://dubai.publicbikesystem.net/ube/gbfs/v1/
2,AR,Ecobici,"Buenos Aires, AR",bike_buenosaires,https://www.buenosaires.gob.ar/ecobici,https://buenosaires.publicbikesystem.net/ube/g...
3,AR,MiBiciTuBici,"Rosario, Santa Fe, AR",biketobike,https://www.mibicitubici.gob.ar/,https://www.mibicitubici.gob.ar/opendata/gbfs....
4,AR,Movo Buenos Aires,"Buenos Aires, AR",movo_bue,https://movo.me/ar,https://gbfs.movo.me/v1/buenosaires/gbfs.json
...,...,...,...,...,...,...
530,US,Wheels Miami,"Miami, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://mia-gbfs.getwheelsapp.com/gbfs.json
531,US,Wheels Orlando,"Orlando, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://orlando-gbfs.getwheelsapp.com/gbfs.json
532,US,Wheels San Diego,"San Diego, CA",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://sandiego-gbfs.getwheelsapp.com/gbfs.json
533,US,Wheels Tallahassee,"Tallahassee, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://tallahassee-gbfs.getwheelsapp.com/gbfs...


In [4]:
#getting data from only US cities
us_data=bigdata_df.loc[(bigdata_df["Country Code"]=="US")]
us_data

,Country Code,Name,Location,System ID,URL,Auto-Discovery URL
331,US,Austin B-cycle,"Austin, TX",bcycle_austin,http://austinbcycle.com,https://gbfs.bcycle.com/bcycle_austin/gbfs.json
332,US,Aventura BCycle,"Aventura, FL",bcycle_aventura,https://aventura.bcycle.com,https://gbfs.bcycle.com/bcycle_aventura/gbfs.json
333,US,Bay Wheels,"San Francisco Bay Area, CA",BA,https://www.baywheels.com/,https://gbfs.baywheels.com/gbfs/gbfs.json
334,US,Bike Chattanooga,"Chattanooga, TN",bike_chattanooga,http://www.bikechattanooga.com/,https://chat.publicbikesystem.net/ube/gbfs/v1/
335,US,BikeLNK,"Lincoln, NE",bcycle_bikelnk,https://bikelnk.bcycle.com,https://gbfs.bcycle.com/bcycle_bikelnk/gbfs.json
...,...,...,...,...,...,...
530,US,Wheels Miami,"Miami, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://mia-gbfs.getwheelsapp.com/gbfs.json
531,US,Wheels Orlando,"Orlando, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://orlando-gbfs.getwheelsapp.com/gbfs.json
532,US,Wheels San Diego,"San Diego, CA",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://sandiego-gbfs.getwheelsapp.com/gbfs.json
533,US,Wheels Tallahassee,"Tallahassee, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://tallahassee-gbfs.getwheelsapp.com/gbfs...


In [5]:
#getting data for bird and lime escooter
data_scoot = us_data.loc[us_data['System ID'].str.contains("spin", case=False)]
len(data_scoot)
#data_scoot
data_scoot

,Country Code,Name,Location,System ID,URL,Auto-Discovery URL
442,US,Spin Aiken,"Aiken, SC",spin aiken,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/aiken/gbfs
443,US,Spin Albuquerque,"Albuquerque, NM",spin albuquerque,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/albuquerque/gbfs
444,US,Spin Alexandria,"Alexandria, VA",spin alexandria,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/alexandria/gbfs
445,US,Spin Ann Arbor,"Ann Arbor, MI",spin ann_arbor,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/ann_arbor/gbfs
446,US,Spin Arkadelphia,"Arkadelphia, VA",spin arkadelphia,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/arkadelphia/gbfs
...,...,...,...,...,...,...
518,US,Spin Washington DC,"Washington, DC",spin washington_dc,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/washington_dc...
519,US,Spin West Sacramento,"West Sacramento, CA",spin west_sacramento,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/west_sacramen...
520,US,Spin White Center,"White Center, WA",spin white_center,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/white_center/...
521,US,Spin Wichita,"Wichita, KS",spin wichita,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/wichita/gbfs


In [20]:
# #getting latitudes and longitudes for city as well as coordinates of bird scooters in each cities
# coord_length=[]
lat_list=[]
lon_list=[]
latlon = []
all_url=data_scoot["Auto-Discovery URL"]
x = 0
y = 0
bike = 0
dict = {}
city_list = []
count_list = []
city_list_coord = []

for i, row in data_scoot.iterrows():
    
    # print(base_url)
    base_url= row['Auto-Discovery URL']
    cities = row['Location']
    print(f'running record {x} on {cities}')
        
    # get URL for location
    result = requests.get(base_url).json()
    result
    # Fixed the jsondecode issue
    #response = requests.get(base_url).json()
    # print(json.dumps(response, indent=4, sort_keys=True))


    # get URL with bike status for location
    base_url1 = result['data']['en']['feeds'][1]['url']
    base_url1

    # get file with location info
    result1 = requests.get(base_url1).json()
    result1

    bike_coord = result1['data']['bikes']
    bike_coord
    print(f'there are {len(bike_coord)} bikes in record {x}')
    city_list.append(cities)
    count_list.append(len(bike_coord))
    
    for bike in bike_coord:
        
        # Add Lat info to list
        lat_list.append([bike][0]['lat'])
        lon_list.append([bike][0]['lon'])
        city_list_coord.append(cities)
    
    
    x = x+1
    time.sleep(2)
 
    
# # # latlon.append(dict)
# # #         x = x +1
# # #         y = y+1
        

        

running record 0 on Aiken, SC
there are 0 bikes in record 0
running record 1 on Albuquerque, NM
there are 0 bikes in record 1
running record 2 on Alexandria, VA
there are 4 bikes in record 2
running record 3 on Ann Arbor, MI
there are 50 bikes in record 3
running record 4 on Arkadelphia, VA
there are 0 bikes in record 4
running record 5 on Arlington, VA
there are 0 bikes in record 5
running record 6 on Asbury Park, NY
there are 0 bikes in record 6
running record 7 on Atlanta, GA
there are 771 bikes in record 7
running record 8 on Austin, TX
there are 0 bikes in record 8
running record 9 on Baltimore, MD
there are 1052 bikes in record 9
running record 10 on Boise, ID
there are 158 bikes in record 10
running record 11 on Brookline, MA
there are 0 bikes in record 11
running record 12 on Charlotte,NC
there are 237 bikes in record 12
running record 13 on Chicago, IL
there are 0 bikes in record 13
running record 14 on Cleveland, OH
there are 321 bikes in record 14
running record 15 on Columb

In [22]:
#to save city level based on Geofence
df1 = pd.DataFrame({'City': city_list, 'Count':count_list})

In [1]:
df1

In [24]:
#to retrieve geocode from Gmaps
lat_list = []
lon_list = [] 

for x in df1["City"]:
    # Target city
    target_city = x
    
    # Build the endpoint URL
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={target_city}&key={gkey}"

    geo_data = requests.get(target_url).json()

    # Extract latitude and longitude
    lat_list.append(geo_data["results"][0]["geometry"]["location"]["lat"])
    lon_list.append(geo_data["results"][0]["geometry"]["location"]["lng"])

In [2]:
#Save city level counts based on geofence to csv
df1["lat"] = lat_list
df1['lon']= lon_list
df1['Operator'] = "Spin"
today = date.today()
df1['Date'] = today
df1.to_csv('Spin_Lat_Lon.csv')
df1

In [3]:
# df1 = pd.DataFrame({'City': city_list, 'Count':count_list})
# df2 = pd.DataFrame({'Lat':lat_list, 'Lon': lon_list, 'City': city_coord})
# df['Operator'] = "Spin"
# today = date.today()
# df['Date'] = today
# df
# df = df.groupby(["City"])
# df.count()
# # df.to_csv('Spin_Lat_Lon.csv')

In [12]:
new_list = list(zip(lat_list,lon_list))
len(new_list)

14993

In [13]:
locations = list(zip(lat_list,lon_list))
locations

[(38.8341, -77.10893),
 (38.83314, -77.08525),
 (38.83407, -77.10895),
 (38.83254, -77.0507),
 (42.2493, -83.72357),
 (42.25733, -83.71391),
 (42.23986, -83.75416),
 (42.24121, -83.75605),
 (42.26679, -83.76948),
 (42.28483, -83.75654),
 (42.27383, -83.77917),
 (42.27287, -83.73298),
 (42.24103, -83.75561),
 (42.27797, -83.7649),
 (42.29025, -83.73385),
 (42.25753, -83.73608),
 (42.25704, -83.73498),
 (42.29477, -83.7364),
 (42.28506, -83.75633),
 (42.28514, -83.72959),
 (42.28225, -83.74943),
 (42.24403, -83.75694),
 (42.25685, -83.73485),
 (42.25843, -83.75027),
 (42.25646, -83.76651),
 (42.27384, -83.7792),
 (42.25968, -83.73553),
 (42.28502, -83.71532),
 (42.25775, -83.69975),
 (42.28071, -83.75699),
 (42.32015, -83.7152),
 (42.24659, -83.75269),
 (42.25137, -83.70957),
 (42.25138, -83.70954),
 (42.29022, -83.75429),
 (42.28405, -83.77581),
 (42.25138, -83.70951),
 (42.2384, -83.74075),
 (42.2734, -83.7259),
 (42.23995, -83.75423),
 (42.26738, -83.77716),
 (42.2899, -83.73117),
 (4

In [14]:

#generating the heat map using the city(lat and lon values) with
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))